## Reduction operation: the sum of the numbers in the range [0, value)

In [1]:
import numpy as np

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

value = 5*10**4

X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

Time taken by reduction operation using a function: 2.62 ms ± 8.27 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
And the result of the sum of numbers in the range [0, value) is: 24995.051343753086

Time taken by reduction operation using numpy.sum(): 7.88 µs ± 8.1 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.sum(): 24995.051343752984 
 
Time taken by reduction operation using numpy.ndarray.sum(): 7.14 µs ± 0.478 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.ndarray.sum(): 24995.051343752984print("Now, the result using numpy.ndarray.sum():", X.sum())




Time taken by reduction operation using a function: 2.62 ms ± 8.27 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
And the result of the sum of numbers in the range [0, value) is: 24995.051343753086

Time taken by reduction operation using numpy.sum(): 7.88 µs ± 8.1 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.sum(): 24995.051343752984 
 
Time taken by reduction operation using numpy.ndarray.sum(): 7.14 µs ± 0.478 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.ndarray.sum(): 24995.051343752984


In [2]:
import numpy as np

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

value = 5*10**7
X = np.random.rand(value).astype(np.float32)


In [3]:
import time

# Función personalizada
start = time.time()
res1 = reduc_operation(X)
end = time.time()
print("reduc_operation:")
print("Resultado:", res1)
print("Tiempo:", end - start, "segundos\n")

# numpy.sum
start = time.time()
res2 = np.sum(X)
end = time.time()
print("np.sum:")
print("Resultado:", res2)
print("Tiempo:", end - start, "segundos\n")

# ndarray.sum()
start = time.time()
res3 = X.sum()
end = time.time()
print("ndarray.sum:")
print("Resultado:", res3)
print("Tiempo:", end - start, "segundos\n")


reduc_operation:
Resultado: 25001724.94909561
Tiempo: 70.57475852966309 segundos

np.sum:
Resultado: 25001724.0
Tiempo: 0.05396842956542969 segundos

ndarray.sum:
Resultado: 25001724.0
Tiempo: 0.05024003982543945 segundos



In [4]:
# a) Paralelismo con multiprocessing

from multiprocessing import Pool

def split_array(arr, n_chunks):
    length = len(arr)
    return [arr[i*length//n_chunks:(i+1)*length//n_chunks] for i in range(n_chunks)]

def parallel_sum(n_procs):
    chunks = split_array(X, n_procs)
    with Pool(n_procs) as p:
        results = p.map(reduc_operation, chunks)
    return sum(results)

for n in [1, 2, 4]:
    start = time.time()
    result = parallel_sum(n)
    end = time.time()
    print(f"{n} procesos - Resultado: {result:.2f} - Tiempo: {end - start:.4f} s")


1 procesos - Resultado: 25001724.95 - Tiempo: 73.2600 s
2 procesos - Resultado: 25001724.95 - Tiempo: 38.3858 s
4 procesos - Resultado: 25001724.95 - Tiempo: 19.4035 s


In [5]:
# b) Optimización con Numba

from numba import njit, prange

@njit
def reduc_operation_njit(A):
    s = 0.0
    for i in range(len(A)):
        s += A[i]
    return s

@njit(parallel=True)
def reduc_operation_parallel(A):
    s = 0.0
    for i in prange(len(A)):
        s += A[i]
    return s


In [6]:
start = time.time()
res_njit = reduc_operation_njit(X)
end = time.time()
print("Numba njit - Resultado:", res_njit)
print("Tiempo:", end - start, "segundos\n")

start = time.time()
res_par = reduc_operation_parallel(X)
end = time.time()
print("Numba parallel=True - Resultado:", res_par)
print("Tiempo:", end - start, "segundos\n")

Numba njit - Resultado: 25001724.94909561
Tiempo: 4.049393653869629 segundos

Numba parallel=True - Resultado: 25001724.949094407
Tiempo: 3.2074410915374756 segundos



In [9]:
# c) Variable value desde línea de comandos

import sys

# Buscar primer argumento que sea un número entero válido
def get_value_from_args(default=5 * 10**7):
    for arg in sys.argv:
        try:
            return int(arg)
        except ValueError:
            continue
    return default

value = get_value_from_args()
print(f"Tamaño del array: {value}")
X = np.random.rand(value).astype(np.float32)

Tamaño del array: 50000000


## Análisis de rendimiento
Los resultados muestran una mejora progresiva en el tiempo de ejecución al aplicar distintas técnicas de paralelismo y optimización:
-	Función original reduc_operation: muy lenta (≈70 s), sirve como referencia de base secuencial.
-	numpy.sum() y ndarray.sum(): extremadamente rápidos (≈0.05 s) gracias a estar implementados en C altamente optimizado.
-	multiprocessing: al duplicar el número de procesos, el tiempo se reduce casi a la mitad, lo que muestra un paralelismo efectivo.
-	Numba: 
         · @njit: reduce el tiempo a ≈4 s solo por compilar la función.
         · @njit(parallel=True): lo reduce aún más a ≈3.2 s usando múltiples hilos internamente.